---
title: "PAC Learning Theory and VC Dimension"
author: "Ram Polisetti"
date: "2024-03-19"
categories: [machine-learning, theory, mathematics, statistics]
tags: [machine-learning-theory, computational-learning, statistical-learning, optimization]
image: "pac_learning.jpg"
description: "A beginner-friendly guide to PAC learning theory and VC dimension with interactive visualizations and practical examples."
jupyter: python3
---


# PAC Learning Theory and VC Dimension

:::{.callout-note}
## Learning Objectives
By the end of this article, you will:
1. Understand PAC learning intuitively and mathematically

2. Visualize VC dimension in practice

3. Calculate sample complexity for real problems

4. Implement PAC learning algorithms

5. Apply VC theory to model selection
:::

## Introduction

Imagine you're teaching a robot to recognize apples . How can you be "probably approximately correct" about its ability to recognize any apple? PAC learning theory gives us the mathematical framework to answer such questions.


In [ ]:
#| code-fold: true
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.svm import SVC
from sklearn.datasets import make_classification, make_circles
from sklearn.model_selection import learning_curve
import time

# Set random seed for reproducibility
np.random.seed(42)

## 1. PAC Learning Visualization

Let's visualize what "probably approximately correct" means:


In [ ]:
#| code-fold: false
def visualize_pac_learning(n_samples=100, noise_level=0.1):
    # Generate synthetic dataset
    X, y = make_circles(n_samples=n_samples, noise=noise_level, factor=0.3)
    
    # Train models with different sample sizes
    sample_sizes = [10, 30, 50, n_samples]
    fig, axes = plt.subplots(2, 2, figsize=(12, 12))
    axes = axes.ravel()
    
    for i, size in enumerate(sample_sizes):
        # Train model on subset
        model = SVC(kernel='rbf')
        idx = np.random.choice(n_samples, size=size, replace=False)
        model.fit(X[idx], y[idx])
        
        # Create grid for decision boundary
        xx, yy = np.meshgrid(np.linspace(X[:, 0].min()-0.5, X[:, 0].max()+0.5, 100),
                            np.linspace(X[:, 1].min()-0.5, X[:, 1].max()+0.5, 100))
        Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
        Z = Z.reshape(xx.shape)
        
        # Plot
        axes[i].contourf(xx, yy, Z, alpha=0.4)
        axes[i].scatter(X[:, 0], X[:, 1], c=y, alpha=0.8)
        axes[i].set_title(f'Training samples: {size}')
    
    plt.tight_layout()
    plt.show()

visualize_pac_learning()

:::{.callout-tip}
## Key Insight
Notice how the decision boundary becomes more stable and accurate as we increase the sample size. This is PAC learning in action!
:::

## 2. VC Dimension Explorer

Let's create an interactive tool to explore VC dimension:


In [ ]:
#| code-fold: false
def explore_vc_dimension(n_points=100):
    def generate_points(n):
        return np.random.rand(n, 2)
    
    def plot_linear_classifier(ax, points, labels):
        if len(points) >= 2:
            model = SVC(kernel='linear')
            try:
                model.fit(points, labels)
                
                # Plot decision boundary
                xx, yy = np.meshgrid(np.linspace(0, 1, 100),
                                   np.linspace(0, 1, 100))
                Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
                Z = Z.reshape(xx.shape)
                ax.contourf(xx, yy, Z, alpha=0.4)
            except:
                pass
        
        # Plot points
        colors = ['red' if l == 0 else 'blue' for l in labels]
        ax.scatter(points[:, 0], points[:, 1], c=colors)
        ax.set_xlim(0, 1)
        ax.set_ylim(0, 1)
    
    # Generate different labelings
    points = generate_points(3)  # Try with 3 points
    all_labels = [[int(i) for i in format(j, f'0{3}b')] 
                 for j in range(2**3)]
    
    fig, axes = plt.subplots(2, 4, figsize=(16, 8))
    axes = axes.ravel()
    
    for i, labels in enumerate(all_labels):
        plot_linear_classifier(axes[i], points, labels)
        axes[i].set_title(f'Labeling {i+1}')
    
    plt.tight_layout()
    plt.show()

explore_vc_dimension()

:::{.callout-note}
## Understanding VC Dimension
The plots above show different possible labelings of 3 points. A linear classifier (VC dimension = 3) can shatter these points in most, but not all configurations.
:::

## Theoretical Foundations

### 1. PAC Learning Framework

The PAC (Probably Approximately Correct) learning framework provides theoretical guarantees for learning algorithms:

$$
P_{S \sim \mathcal{D}^m}(\text{error}_\mathcal{D}(h_S) \leq \epsilon) \geq 1-\delta
$$

Where:
- $\epsilon$ is the accuracy parameter (how close to perfect)

- $\delta$ is the confidence parameter (how sure we are)

- $m$ is the sample size

- $h_S$ is the learned hypothesis

### 2. Sample Complexity

The fundamental bound for sample complexity:

$$
m \geq \frac{1}{\epsilon}\left(\ln|\mathcal{H}| + \ln\frac{1}{\delta}\right)
$$

For the realizable case (when perfect classification is possible):

$$
m \geq \frac{1}{\epsilon}\left(\ln\frac{1}{\delta}\right)
$$

### 3. VC Dimension Theory

The VC dimension of a hypothesis class $\mathcal{H}$ is the largest number of points that can be shattered (assigned any possible labeling) by $\mathcal{H}$.

Growth function:

$$
\Pi_\mathcal{H}(m) = \max_{x_1,...,x_m \in \mathcal{X}}|\{(h(x_1),...,h(x_m)): h \in \mathcal{H}\}|
$$

Sauer's Lemma:

$$
\text{If VC}(\mathcal{H}) = d, \text{ then } \Pi_\mathcal{H}(m) \leq \sum_{i=0}^d \binom{m}{i}
$$

### 4. Generalization Bounds

The fundamental theorem of learning theory:

$$
P(\sup_{h \in \mathcal{H}}|\text{error}_\mathcal{D}(h) - \widehat{\text{error}}_S(h)| > \epsilon) \leq 4\Pi_\mathcal{H}(2m)\exp(-\frac{m\epsilon^2}{8})
$$

Sample complexity in terms of VC dimension:

$$
m = O\left(\frac{d}{\epsilon^2}\ln\frac{1}{\epsilon} + \frac{1}{\epsilon^2}\ln\frac{1}{\delta}\right)
$$

:::{.callout-note}
## Key Insight
The VC dimension ($d$) appears in the sample complexity bound, showing how model complexity affects learning guarantees.
:::

## PAC Learning Framework

### 1. Basic Definitions

Let's make PAC learning concrete with an example:


In [ ]:
#| code-fold: false
class PACLearner:
    def __init__(self, epsilon=0.1, delta=0.05):
        self.epsilon = epsilon  # accuracy parameter
        self.delta = delta    # confidence parameter
        self.model = None
    
    def required_samples(self, vc_dim):
        """Calculate required sample size using VC bound"""
        return int(np.ceil((8/self.epsilon) * 
                         (2*vc_dim * np.log2(16/self.epsilon) + 
                          np.log2(2/self.delta))))
    
    def fit(self, X, y):
        """Train model with PAC guarantees"""
        n_samples = len(X)
        required = self.required_samples(vc_dim=3)  # for linear classifier
        
        if n_samples < required:
            print(f"Warning: Need at least {required} samples for PAC guarantees")
        
        self.model = SVC(kernel='linear')
        self.model.fit(X, y)
        return self
    
    def predict(self, X):
        return self.model.predict(X)

# Example usage
X, y = make_classification(n_samples=1000, n_features=2, n_redundant=0,
                          n_informative=2, random_state=42)
learner = PACLearner(epsilon=0.1, delta=0.05)
print(f"Required samples: {learner.required_samples(vc_dim=3)}")
learner.fit(X, y)

listing:
  contents: "../**/index.qmd"
  type: default
  fields: [title, description, date, author]
  sort: "date desc"
  max-items: 2
  filter-ui: false
  categories: false
  include-in-header: false
  feed: true
  date-format: "MMMM D, YYYY"

listing:
  contents: "../**/index.qmd"
  type: default
  fields: [title, description, date, author]
  sort: "date desc"
  max-items: 2
  filter-ui: false
  categories: false
  include-in-header: false
  feed: true
  date-format: "MMMM D, YYYY"

---
:::
:::